#Intent Classification using LSTM 
This use-case provides a demo of how LSTM can be used for Intent classification in texts. 

##Workflow:

1.   Understanding the problem
2.   Reading the data and understanding it
3.   Data Preprocessing
4.   Build LSTM model
5.   Train & Evaluate the model

##1. Understanding the problem
Intent Classification is the automated association of text to a specific intention. For example: Let's say you are writing an email to one of the Airlines and the text of the same is 'Can you please cancel my ticket with PNR 123456'. The intent of the customer here is 'Cancellation of Air Ticket'.

The idea of this use case to introduce the concept of Intent classification and how can LSTM be used to solve this. 

###Import the necessary libraries
Please load the following packages before you proceed further. 


In [1]:
import numpy as np 
import pandas as pd 
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.initializers import he_uniform, glorot_uniform
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##2. Collecting the Data
The ATIS(Air Travel Information System) data is a rich corpus that contains natural language text used by general public to book flight tickets, enquire about flight timings, prices etc. 

The Train and test data can be downloaded from https://www.kaggle.com/hassanamin/atis-airlinetravelinformationsystem

There are 2 columns in each of the above datasets. First column is 'target' which is the output we will be classifying and second column is 'text' which is the user input asking for queries related to flights. 

Basically 'target' is the intent of the customer. 

In [2]:
!wget "https://www.dropbox.com/s/8pizebkowpjlevm/intent.zip"
!unzip intent.zip

--2021-10-03 15:51:06--  https://www.dropbox.com/s/8pizebkowpjlevm/intent.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8pizebkowpjlevm/intent.zip [following]
--2021-10-03 15:51:06--  https://www.dropbox.com/s/raw/8pizebkowpjlevm/intent.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd7563dd39baf1d27303778abac.dl.dropboxusercontent.com/cd/0/inline/BXWpUC7m8dTguihrJtYyYaBoRS-dpiYAVum3g6DQtLniyIW9Xsyr_d0RKpetHE6rwPeG8nkeRYiuimuCaqM1yan2Tb-j8SXFrmsmV4aEe6h6ic3Z2TGhmZXFyA7DlJ0HVgyuQtpJ3vTHsp2ohmgDkKrf/file# [following]
--2021-10-03 15:51:06--  https://ucd7563dd39baf1d27303778abac.dl.dropboxusercontent.com/cd/0/inline/BXWpUC7m8dTguihrJtYyYaBoRS-dpiYAVum3g6DQtLniyIW9Xsyr_d0RKpetHE6rwPeG8nkeRYiuimuCaqM1yan2Tb-j8

In [3]:
#Read the train and test datasets with column names as target and text
train= pd.read_csv('atis_intents_train.csv',
                       names= ["target", "text"])

test= pd.read_csv('atis_intents_test.csv',
                       names= ["target", "text"])

####This is how the data looks like

In [4]:
train.head(10) #Get Top 10 rows from train dataset

target                                               text
0          atis_flight   i want to fly from boston at 838 am and arriv...
1          atis_flight   what flights are available from pittsburgh to...
2     atis_flight_time   what is the arrival time in san francisco for...
3         atis_airfare            cheapest airfare from tacoma to orlando
4         atis_airfare   round trip fares from pittsburgh to philadelp...
5          atis_flight   i need a flight tomorrow from columbus to min...
6        atis_aircraft   what kind of aircraft is used on a flight fro...
7          atis_flight   show me the flights from pittsburgh to los an...
8          atis_flight              all flights from boston to washington
9  atis_ground_service   what kind of ground transportation is availab...

 Check the number of intents.

In [5]:
train['target'].value_counts() #Get counts of different types of target variable in train data. We will not be using this anywhere but it is just for the overview of the data.  

atis_flight            3666
atis_airfare            423
atis_ground_service     255
atis_airline            157
atis_abbreviation       147
atis_aircraft            81
atis_flight_time         54
atis_quantity            51
Name: target, dtype: int64

### 3. Preprocessing the Data
We will be doing the following preprocessing steps to get the desired format of the data. 

1. Perform One Hot Encoding on the target variable of both train & test datasets.
2. Convert the text into lower case.
3. Tokenize the words.
4. Remove stop words.
5. Perform stemming & normalization.
6. Convert texts into sequences.
7. Pad the sequences.


In [6]:
encode_target= oneHot().fit(np.array(train.target).reshape(-1,1)) #We perform one hot encoding on the target variable to convert into a matrix of 0s and 1s. 

Perform One Hot Encoding on the target variable. The output of this step would be an array with 0s and 1s. 

In [7]:
train_target_encoded= encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()

Convert text to lower case

In [8]:
train["text"]= train.text.map(lambda l: l.lower())
test["text"]= test.text.map(lambda l: l.lower())

Next step is to tokenize the text. We use word_tokenize function from nltk library for this purpose. 

In [9]:
train["text"]= train.text.map(word_tokenize)
test["text"]= test.text.map(word_tokenize)

In [10]:
#Output of the above exercise looks like this
train["text"]

0       [i, want, to, fly, from, boston, at, 838, am, ...
1       [what, flights, are, available, from, pittsbur...
2       [what, is, the, arrival, time, in, san, franci...
3          [cheapest, airfare, from, tacoma, to, orlando]
4       [round, trip, fares, from, pittsburgh, to, phi...
                              ...                        
4829    [what, is, the, airfare, for, flights, from, d...
4830    [do, you, have, any, flights, from, denver, to...
4831    [which, airlines, fly, into, and, out, of, den...
4832    [does, continental, fly, from, boston, to, san...
4833    [is, there, a, delta, flight, from, denver, to...
Name: text, Length: 4834, dtype: object

Eliminate stop words. 'english' dictionary from nltk.corpus library is used for this purpose. We also remove punctuation along with the removal of stop words.

In [11]:
def clean_data_rm_stop(strings, stop_list): # tokenized words
    sw= [str for str in strings if str not in stop_list] # list comprehension
    return sw

stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

train["text"]= train.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))
test["text"]= test.text.map(lambda dataframe: clean_data_rm_stop(dataframe, rm_punc_stop))

In [12]:
rm_punc_stop

['.',
 '#',
 '&',
 '"',
 ';',
 '`',
 '_',
 '|',
 '+',
 '%',
 '>',
 '[',
 '/',
 '$',
 '?',
 '=',
 ':',
 '{',
 '\\',
 ',',
 '(',
 '@',
 '*',
 '-',
 ']',
 '~',
 '^',
 "'",
 '!',
 '<',
 '}',
 ')',
 'i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from

In [13]:
train["text"]

0       [want, fly, boston, 838, arrive, denver, 1110,...
1       [flights, available, pittsburgh, baltimore, th...
2       [arrival, time, san, francisco, 755, flight, l...
3                    [cheapest, airfare, tacoma, orlando]
4       [round, trip, fares, pittsburgh, philadelphia,...
                              ...                        
4829    [airfare, flights, denver, pittsburgh, delta, ...
4830            [flights, denver, baltimore, via, dallas]
4831                              [airlines, fly, denver]
4832    [continental, fly, boston, san, francisco, sto...
4833              [delta, flight, denver, san, francisco]
Name: text, Length: 4834, dtype: object

Stemming & Normalizing


*   Stemming helps in reducing the word to the root form.
*   Normalizing is the process of transforming text into a standard form. Eg: Gud will be converted to good etc.




In [14]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["text"]= train.text.map(lambda s: [stem_func.stem(x) for x in s])
train["text"]= train.text.apply(normalize)

test["text"]= test.text.map(lambda s: [stem_func.stem(x) for x in s])
test["text"]= test.text.apply(normalize)

In [15]:
train.head()

target                                               text
0       atis_flight         want fli boston 838 arriv denver 1110 morn
1       atis_flight     flight avail pittsburgh baltimor thursday morn
2  atis_flight_time  arriv time san francisco 755 flight leav washi...
3      atis_airfare                     cheapest airfar tacoma orlando
4      atis_airfare  round trip fare pittsburgh philadelphia 1000 d...

In [49]:
max_len = 0
for i in train.text :
  if len(i) == 178 :
    print(i)
  if len(i) > max_len :
    max_len = len(i)
print(max_len)

want travel kansa citi chicago round trip leav wednesday june sixteenth arriv chicago around 7 o'clock even return next day arriv kansa citi around 7 o'clock even airlin fli rout
178


Tokenize

In [16]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
#Consider top 10000 words
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.text)

In [17]:
train.text[0]

'want fli boston 838 arriv denver 1110 morn'

In [18]:
tokenized_train_data[0]

[42, 19, 2, 296, 17, 5, 376, 21]

In [19]:
text_tokenizer.word_index

{"'d": 28,
 "'ll": 314,
 "'m": 131,
 "'re": 268,
 "'s": 74,
 "'ve": 571,
 '0900': 619,
 '1': 229,
 '10': 114,
 '100': 456,
 '1000': 180,
 '1020': 590,
 '1024': 455,
 '1026': 492,
 '1030': 473,
 '1039': 502,
 '1045': 550,
 '1055': 559,
 '1059': 533,
 '106': 481,
 '1083': 618,
 '11': 277,
 '110': 527,
 '1100': 487,
 '1110': 376,
 '1115': 324,
 '1130': 474,
 '1133': 623,
 '1145': 415,
 '1158': 540,
 '12': 137,
 '1200': 609,
 '1205': 568,
 '1209': 638,
 '1222': 409,
 '1230': 612,
 '124': 493,
 '1245': 325,
 '1288': 460,
 '1291': 357,
 '130': 543,
 '1300': 640,
 '139': 486,
 '150': 526,
 '1500': 637,
 '1505': 597,
 '1600': 513,
 '163': 649,
 '1700': 391,
 '1765': 395,
 '1850': 596,
 '19': 484,
 '1940': 645,
 '1991': 198,
 '1992': 326,
 '1993': 593,
 '2': 181,
 '200': 504,
 '201': 405,
 '21': 358,
 '210': 512,
 '2100': 333,
 '212': 627,
 '2134': 393,
 '2153': 476,
 '217': 420,
 '225': 539,
 '229': 615,
 '230': 317,
 '257': 461,
 '269': 435,
 '270': 371,
 '271': 534,
 '281': 292,
 '296': 367,

Then, we pad the sequences

In [20]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library -> Consider only first 20 words for processing
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")

In [21]:
tokenized_train_data[:5]

[[42, 19, 2, 296, 17, 5, 376, 21],
 [1, 37, 8, 10, 45, 21],
 [17, 102, 4, 6, 377, 1, 12, 18],
 [32, 212, 142, 91],
 [36, 30, 16, 8, 11, 180, 134]]

In [22]:
train_data[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  42,
         19,   2, 296,  17,   5, 376,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   1,  37,   8,  10,  45,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  17,
        102,   4,   6, 377,   1,  12,  18],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,  32, 212, 142,  91],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         36,  30,  16,   8,  11, 180, 134]], dtype=int32)

In [35]:
train_data.shape

(4834, 20)

Let's build a 3 dim array. The dimensions are samples, steps and unique words.

In [24]:
text_tokenizer.word_index.keys().__len__()

654

In [25]:
len(text_tokenizer.word_index)

654

In [26]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [27]:
trans_matrix_train.shape

(4834, 20, 654)

In [28]:
len(text_tokenizer.word_index.keys()) # Number of words in vocabulory

654

### 4. Build LSTM Model

Build a class lstm_model_class that has three methods(methods are similar to functions) in a class, they are:



1.   Build a lstm model 
2.   Train the created lstm model on the train data
3.   Predict the output on the train data

Note: Building a Class with 3 methods helps in tying all these three functions to the same object at the same instance. 

Process of Building a LSTM Model:



1.   Build an embedded layer with dimensions as number of steps and input dimensions. 
2.   Build an LSTM layer with number of steps equal to memory units.  
3.   Then, build a dense layer which is fully connected layer that represents a matrix vector multiplication. 
4.   Apply the relu function after normalization and scaling of the activations. This is the standard activation function used. 
5.   Finally, build the output layer.

Note: 
1. activation function for multi-class classification problem - softmax
2. loss function is categorical cross entropy. 
3. performace metric - Area under the curve(AUC)
4. optimizer would be the Adam optimizer. 





In [29]:
class lstm_model_class(object):
    def build_lstm_model(self,input_dimensions, op_shape, num_steps, dropout_rate, kernel_reg, bias_reg):
      ip_layer= Input(shape= (num_steps, input_dimensions)) #Define embedded layer with shape as number of steps and input dimensions. Note that both these are input variables to the model.
        
      lstm_model= LSTM(units= num_steps)(ip_layer) #Make the LSTM layer with number of steps as memory units # 20
      
      dense_layer_1= Dense(op_shape, kernel_initializer= he_uniform(), #he_uniform draws samples in uniform distribution with -inf to +inf as range.
                   bias_initializer= "zeros", 
                   kernel_regularizer= l2(l= kernel_reg),
                   bias_regularizer= l2(l= bias_reg))(lstm_model) # Create the  Dense layer which is the regular deeply connected layer 
      int_layer= BatchNormalization()(dense_layer_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      int_layer= relu(int_layer) #This applies the rectified linear unit activation function
      int_layer= Dropout(rate= dropout_rate)(int_layer) #Dropout is used to define Dropout layer that sets input units to 0 with a frequency. Here it is dropout_rate
      
      output_1= Dense(op_shape, kernel_initializer= glorot_uniform(), #glorot_uniform draws samples in uniform distribution with stddev = sqrt(2 / (fan_in + fan_out)) fan_in is num of units in weight tensor and fan_out is num of output units
             bias_initializer= "zeros", 
             kernel_regularizer= l2(l= kernel_reg), 
             bias_regularizer= l2(l= bias_reg))(int_layer) # Create another dense layer which is the output of the model.
      output_1= BatchNormalization()(output_1) #Normalize and scale activations of the dense layer with BatchNormalization function
      final_output= softmax(output_1, axis= 1) 
        
      loss_func= cce() # Since it is a multi-class classification problem, categorical crossentropy(cce) is used as the loss function
      perf_metrics= AUC() #our performance metric will be area under the curve
      optimizer= Adam() #we shall use Adam optimizer as our optimizer
      self.final_model= Model(inputs= [ip_layer], outputs= [final_output]) #Build the model with input and output layers
      self.final_model.compile(optimizer= optimizer, loss= loss_func, metrics= [perf_metrics]) #Compiling the keras model

    def train_lstm_model(self,x, y, valid_split, ep):
      self.final_model.fit(x, y, validation_split= valid_split, epochs= ep) #Create the train model


    def predict_lstm_model(self,x):
      return self.final_model.predict(x)    #Create the predict model


Build the model on the necessary inputs.
We define the number of steps as the , output shape and input dimensio appropriately. 

In [30]:
steps= trans_matrix_train.shape[1] #Define the number of steps is usually the number of steps in the train data. # 20
input_dim= trans_matrix_train.shape[2] #Input dimension. Number of unique words in the train data #
output_shape= train_target_encoded.shape[1] #Output shape. Usually the same number as the number of classes in the target variable. Here we have 8. 
final_model= lstm_model_class()
final_model.build_lstm_model(input_dimensions= input_dim,
                  op_shape= output_shape,
                  num_steps= steps, 
                  dropout_rate= 0.5, # Meaning 1 in 2 inputs will be randomly executed. 
                  bias_reg= 0.3, # Reduce the bias in the model
                  kernel_reg= 0.3) #Reduce the weights excluding bias. 



### 5. Train & Evaluate the model
In our last step, we will train and evaluate the model and check the performance metrics. 

In [31]:
final_model.train_lstm_model(trans_matrix_train, train_target_encoded,
           0.2, 60) #Model takes train data, train target variable, validation split(here it is 80:20) and number of epochs. 

Epoch 1/60
121/121 [==============================] - 6s 19ms/step - loss: 7.7260 - auc: 0.5985 - val_loss: 5.8737 - val_auc: 0.9085
Epoch 2/60
121/121 [==============================] - 1s 11ms/step - loss: 4.4736 - auc: 0.8443 - val_loss: 3.6407 - val_auc: 0.9556
Epoch 3/60
121/121 [==============================] - 1s 11ms/step - loss: 2.6733 - auc: 0.9268 - val_loss: 2.3648 - val_auc: 0.9301
Epoch 4/60
121/121 [==============================] - 1s 11ms/step - loss: 1.7337 - auc: 0.9549 - val_loss: 1.6768 - val_auc: 0.9415
Epoch 5/60
121/121 [==============================] - 1s 11ms/step - loss: 1.2425 - auc: 0.9691 - val_loss: 1.3032 - val_auc: 0.9170
Epoch 6/60
121/121 [==============================] - 1s 11ms/step - loss: 0.9805 - auc: 0.9771 - val_loss: 2.8071 - val_auc: 0.3419
Epoch 7/60
121/121 [==============================] - 1s 11ms/step - loss: 0.8267 - auc: 0.9811 - val_loss: 6.8234 - val_auc: 0.2738
Epoch 8/60
121/121 [==============================] - 1s 11ms/step - 

In [32]:
pred_train= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_train)) #Predict on the train matrix and look at the performance
print(classification_report(train.target, pred_train)) #Print the classification report

                     precision    recall  f1-score   support

  atis_abbreviation       0.98      0.99      0.99       147
      atis_aircraft       0.57      1.00      0.72        81
       atis_airfare       0.98      0.99      0.99       423
       atis_airline       0.90      0.98      0.94       157
        atis_flight       1.00      1.00      1.00      3666
   atis_flight_time       1.00      0.54      0.70        54
atis_ground_service       1.00      0.99      0.99       255
      atis_quantity       0.00      0.00      0.00        51

           accuracy                           0.98      4834
          macro avg       0.80      0.81      0.79      4834
       weighted avg       0.98      0.98      0.98      4834



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 and weighted avg are excellent. We can now move to implement this model on test data and see how it is performing.

In [33]:
pred_test= encode_target.inverse_transform(final_model.predict_lstm_model(trans_matrix_test)) #Predict on the test data
print(classification_report(test.target, pred_test)) #Print the classification report

                     precision    recall  f1-score   support

  atis_abbreviation       1.00      1.00      1.00        33
      atis_aircraft       0.47      1.00      0.64         9
       atis_airfare       0.96      1.00      0.98        48
       atis_airline       0.86      1.00      0.93        38
        atis_flight       1.00      0.98      0.99       632
   atis_flight_time       0.00      0.00      0.00         1
atis_ground_service       1.00      0.94      0.97        36
      atis_quantity       0.00      0.00      0.00         3

           accuracy                           0.97       800
          macro avg       0.66      0.74      0.69       800
       weighted avg       0.98      0.97      0.97       800



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 and weighted avg are excellent. We can settle with this model. 

### Next Steps


*   Try changing some of the parameters and see if there could be any change in the performance metrics. 



In [34]:
# Normal pre-processing
# One Hot Representaion of the data

# Pretrained Embedding layer